In [ ]:

# CareerSync Fine-tuning - Load Dataset
import json
import os
from google.colab import files

# Load the CareerSync dataset
file = json.load(open("careersync_dataset.json", "r"))
print("Sample data:")
print(json.dumps(file[0], indent=2))
print(f"\nTotal samples: {len(file)}")

In [ ]:
# CareerSync Fine-tuning - Install Dependencies
!pip uninstall -y unsloth peft

!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

In [ ]:
from datasets import Dataset

def format_prompt(example):
    # Format specifically for resume vs job description analysis
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

# Format the data
formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

print(f"Formatted dataset size: {len(dataset)}")
print("Sample formatted prompt:")
print(formatted_data[0][:500] + "..." if len(formatted_data[0]) > 500 else formatted_data[0])

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for CareerSync dataset
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=5,  # Increased for better learning with small dataset
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=5,  # More frequent logging for small dataset
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to=None,  # Disable wandb logging
    ),
)

print("Starting training...")

In [ ]:
# Training arguments optimized for CareerSync dataset
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=5,  # Increased for better learning with small dataset
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=5,  # More frequent logging for small dataset
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to=None,  # Disable wandb logging
    ),
)

print("Starting training...")

In [ ]:
# Train the model
trainer_stats = trainer.train()
print("Training completed!")

# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
# Test with CareerSync-specific prompt
test_input = """Analyze resume vs job description:
RESUME: Python Developer with 2 years experience, Django, Flask, PostgreSQL, Git. Built REST APIs and web applications.
JOB: Backend Developer position. Python, FastAPI, microservices, 3+ years experience, AWS knowledge preferred."""

messages = [
    {"role": "user", "content": test_input},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

In [ ]:
# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=512,  # Increased for detailed analysis
    use_cache=True,
    temperature=0.3,  # Lower temperature for more consistent analysis
    do_sample=True,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print("\n" + "="*50)
print("MODEL RESPONSE:")
print("="*50)
print(response)

In [ ]:
# Test with another example
test_input_2 = """Analyze resume vs job description:
RESUME: Marketing Coordinator, 1 year experience, social media management, content creation, Google Analytics, Canva.
JOB: Digital Marketing Manager, 4+ years experience, SEO, PPC campaigns, marketing automation, team leadership."""

messages_2 = [
    {"role": "user", "content": test_input_2},
]

inputs_2 = tokenizer.apply_chat_template(
    messages_2,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs_2 = model.generate(
    input_ids=inputs_2,
    max_new_tokens=512,
    use_cache=True,
    temperature=0.3,
    do_sample=True,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

response_2 = tokenizer.batch_decode(outputs_2)[0]
print("\n" + "="*50)
print("MODEL RESPONSE 2:")
print("="*50)
print(response_2)

In [ ]:
# Save the model in GGUF format for deployment
print("Saving model in GGUF format...")
model.save_pretrained_gguf("careersync_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
# Download the model file
gguf_files = [f for f in os.listdir("careersync_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("careersync_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)
else:
    print("No GGUF files found!")

print("\nTraining completed! Your CareerSync model is ready for deployment.")
print("The model can now analyze resumes against job descriptions and provide:")
print("- Match scores")
print("- Matched skills")
print("- Missing skills") 
print("- Recommendations")
print("- Role alignment assessment")